In [3]:
!pip install -q transformers datasets accelerate evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.2 MB/s eta 0:00:00


In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, classification_report

In [5]:
# 1.Load and inspect the dataset
# Load the emotion classification dataset from Hugging Face
dataset = load_dataset("emotion")

# Print dataset sizes to understand the data split
print(f"Train samples: {len(dataset['train'])}")
print(f"Test samples: {len(dataset['test'])}")
print(f"Validation samples: {len(dataset['validation'])}")

# Inspect a single training sample
print("\nSample:")
print(dataset['train'][0])

# Extract label names for later use
label_names = dataset['train'].features['label'].names
print(f"\nLabels: {label_names}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

split/train-00000-of-00001.parquet:   0%|          | 0.00/1.03M [00:00<?, ?B/s]

split/validation-00000-of-00001.parquet:   0%|          | 0.00/127k [00:00<?, ?B/s]

split/test-00000-of-00001.parquet:   0%|          | 0.00/129k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Train samples: 16000
Test samples: 2000
Validation samples: 2000

Sample:
{'text': 'i didnt feel humiliated', 'label': 0}

Labels: ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']


In [6]:
# 2.Model and tokenizer selection
# Use a lightweight BERT based model suitable for text classification
model_name = "distilbert-base-uncased"
num_labels = len(dataset['train'].features['label'].names)


# Load tokenizer and model with correct number of output labels
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# 3.Text tokenization
# Tokenization function converts raw text into model input format
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)

# Apply tokenization to the entire dataset
# Remove raw text column since the model only needs tokenized inputs
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=['text'])

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [32]:
# 4.Training configuration
# Define training arguments for fine tuning
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,  # Reduced batch size for better learning
    per_device_eval_batch_size=16,
    num_train_epochs=8,  # Increased from 3 to 8
    weight_decay=0.01,
    logging_steps=50,
    load_best_model_at_end=True,
    warmup_steps=500,  # Added warmup
    save_total_limit=2,  # Save only best 2 models
)

# 5.Evaluation metrics
# Custom metric function for accuracy and F1 score
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {
        'accuracy': accuracy_score(labels, predictions),
        'f1': f1_score(labels, predictions, average='weighted')
    }

In [33]:
# 6.Trainer initialization
# Hugging Face Trainer handles training and evaluation loop
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    compute_metrics=compute_metrics,
)

# 7.Model training
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.143900,0.205300,0.930000,0.930659
2,0.131500,0.186097,0.933500,0.933317
3,0.091200,0.196630,0.939000,0.939214
4,0.064100,0.276652,0.936500,0.937181
5,0.035800,0.322361,0.935000,0.934258
6,0.019400,0.329581,0.939000,0.938815
7,0.008300,0.328991,0.941000,0.940972
8,0.001200,0.331263,0.944000,0.943968


TrainOutput(global_step=8000, training_loss=0.06670366607792676, metrics={'train_runtime': 1444.1185, 'train_samples_per_second': 88.635, 'train_steps_per_second': 5.54, 'total_flos': 4239259140096000.0, 'train_loss': 0.06670366607792676, 'epoch': 8.0})

In [35]:
# 8.Final evaluation on test set
# Fixed classification report
results = trainer.evaluate(tokenized_datasets['test'])
print(f"Accuracy: {results['eval_accuracy']:.4f}")
print(f"F1 Score: {results['eval_f1']:.4f}")

# Generate predictions
predictions = trainer.predict(tokenized_datasets['test'])
pred_labels = np.argmax(predictions.predictions, axis=1)
true_labels = tokenized_datasets['test']['label']

# Check unique labels in test set
unique_labels = sorted(set(true_labels))
print(f"\nUnique labels in test set: {unique_labels}")
print(f"Number of unique labels: {len(unique_labels)}")

# Create label names for only the classes present in test set
present_label_names = [label_names[i] for i in unique_labels]

print("\n" + classification_report(
    true_labels,
    pred_labels,
    labels=unique_labels,
    target_names=present_label_names
))

Accuracy: 0.9270
F1 Score: 0.9268

Unique labels in test set: [0, 1, 2, 3, 4, 5]
Number of unique labels: 6

              precision    recall  f1-score   support

       anger       0.97      0.97      0.97       581
     disgust       0.94      0.95      0.94       695
        fear       0.85      0.80      0.82       159
         joy       0.93      0.92      0.93       275
     sadness       0.88      0.90      0.89       224
    surprise       0.75      0.74      0.75        66

    accuracy                           0.93      2000
   macro avg       0.89      0.88      0.88      2000
weighted avg       0.93      0.93      0.93      2000



In [37]:
# 9.Inference function
# Function for predicting emotion from raw text
def predict_emotion(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

    # Move inputs to GPU if available
    if torch.cuda.is_available():
        inputs = {k: v.cuda() for k, v in inputs.items()}

    # Disable gradient calculation for inference
    with torch.no_grad():
        outputs = model(**inputs)

    # Extract predicted class and confidence
    predicted_class = torch.argmax(outputs.logits, dim=1).item()
    confidence = torch.softmax(outputs.logits, dim=1)[0][predicted_class].item()
    return label_names[predicted_class], confidence

In [45]:
# Test the chatbot with clearer emotions
test_cases = [
    "I am so angry right now!",
    "This is absolutely disgusting!",
    "I am terrified and very scared!",
    "I am so happy and excited today!",
    "I feel so sad and depressed.",
    "Wow, that was completely unexpected!",
]

for text in test_cases:
    emotion, confidence = predict_emotion(text)
    print(f"Text: {text}")
    print(f"→ {emotion.upper()} ({confidence:.1%})\n")

Text: I am so angry right now!
→ ANGER (99.3%)

Text: This is absolutely disgusting!
→ ANGER (97.1%)

Text: I am terrified and very scared!
→ FEAR (99.2%)

Text: I am so happy and excited today!
→ JOY (99.3%)

Text: I feel so sad and depressed.
→ SADNESS (99.7%)

Text: Wow, that was completely unexpected!
→ SURPRISE (67.9%)



In [13]:
# Save model and tokenizer for later use or deployment
trainer.save_model("./trained_model")
tokenizer.save_pretrained("./trained_model")

('./trained_model/tokenizer_config.json',
 './trained_model/special_tokens_map.json',
 './trained_model/vocab.txt',
 './trained_model/added_tokens.json',
 './trained_model/tokenizer.json')

In [54]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Trained this model to classify emotions from text
MODEL_PATH = "/content/a/trained_model"

print("Loading model...")

# Tokenizer converts text into input IDs that the model understands
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, local_files_only=True)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH, local_files_only=True)


label_names = ["sadness", "joy", "love", "anger", "fear", "surprise"]

# Move model to the chosen device and set it to evaluation mode
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval() # evaluation mode disables dropout
print(f"Model loaded! Using: {device}")

Loading model...
Model loaded! Using: cuda


In [28]:
def predict_emotion(text):
    if not text.strip():
        return None, 0.0

    # Tokenize the input text for the model
    # truncation/padding ensures consistent input size
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Run the model without computing gradients (inference mode)
    with torch.no_grad():
        outputs = model(**inputs)

    # Convert logits to probabilities
    probabilities = torch.softmax(outputs.logits, dim=1)[0]
    # Get the predicted class index
    predicted_class = torch.argmax(probabilities).item()
    # Get confidence of the prediction
    confidence = probabilities[predicted_class].item()

    return label_names[predicted_class], confidence

In [52]:
# These make the chatbot feel human-like
responses = {
    "sadness": "I'm sorry you're feeling down. I hope things get better soon.😔",
    "joy": "That sounds wonderful! I'm glad you're feeling happy!😊",
    "love": "That's so sweet! Love is a beautiful thing!❤️",
    "anger": "I can understand your frustration. Take a deep breath.😤",
    "fear": "That sounds concerning. I hope everything works out for you.🤗",
    "surprise": "Wow, that's quite unexpected!😲"
}

In [53]:
# Display a welcome message and instructions to the user
print("-" * 60)
print("EMOTION DETECTION CHATBOT - NLU PROJECT")
print("-" * 60)
print("\nType your message to analyze emotions!")
print("Type 'exit' to stop\n")

while True:
    user_input = input("You: ").strip()

    if user_input.lower() in ["exit", "quit", "bye"]:
        print("\nBot: Goodbye! 👋")
        break

    if not user_input:
        continue

    emotion, confidence = predict_emotion(user_input)
    bot_response = responses.get(emotion, "I understand.")

    print(f"Emotion: {emotion.upper()} | Confidence: {confidence:.1%}")
    print(f"Bot: {bot_response}\n")

EMOTION DETECTION CHATBOT - NLU PROJECT

Type your message to analyze emotions!
Type 'exit' to stop

You: i like the smell of rose
Emotion: LOVE | Confidence: 64.8%
Bot: That's so sweet! Love is a beautiful thing!❤️

You: I am afraid of storms.
Emotion: FEAR | Confidence: 99.1%
Bot: That sounds concerning. I hope everything works out for you.🤗

You: wow, that's amazing
Emotion: SURPRISE | Confidence: 64.8%
Bot: Wow, that's quite unexpected!😲

You: exit

Bot: Goodbye! 👋
